In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorboard

In [2]:

img_height = 150
img_width =150
def image_processing(path):
    folder_path = path#"./simpsons/simpsons_dataset/"
    
    data_path = os.listdir(folder_path)
    X_image = []
    Y_output = []
    Y_label = []
    count =0
    #optimize it
    
    for category in data_path:
        print(category)
        path = os.path.join(f"{folder_path}",category)

        Y_label.append(category)
        for name in os.listdir(path):

            image = os.path.join(f"{path}/",name)
            image = cv2.imread(image)
            if image is None:
                print("Wrong Path")
            else:
                resize_image = cv2.resize(image,(150,150))
                #gray_scale = cv2.cvtColor(resize_image,cv2.COLOR_BGR2GRAY)
            X_image.append(resize_image)
            Y_output.append(count)
        count +=1 
        
    X = np.array(X_image)
    Y = np.array(Y_output)
    X_final = X/255
  
    
    return X_final,Y,Y_label,count
    
    

In [4]:
path = input("Enter folder path: ")
x_final,Y, Y_label,count = image_processing(path)


x_train,x_test,y_train,y_test = train_test_split(x_final,Y,random_state=0)

Enter folder path: ./simpsons/simpsons_dataset/
abraham_grampa_simpson
agnes_skinner
apu_nahasapeemapetilon
barney_gumble
bart_simpson
carl_carlson
charles_montgomery_burns
chief_wiggum
cletus_spuckler


In [13]:
# using sequential API

model = Sequential([
  
    layers.Conv2D(128,4,padding='same',activation ='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(64,4,padding='same',activation = 'relu'),
    layers.MaxPool2D(),
    layers.Conv2D(64,4,padding='same',activation = 'relu'),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(count)
    
    
])

In [14]:
#model = CustomModel(count)
#tensorboard_callbacks = keras.callbacks.TensorBoard(log_dir="logs_simpsons/",histogram_freq=1)
model.compile(
          loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics   = ['accuracy'],
          optimizer = tf.keras.optimizers.Adam())

In [15]:
model.fit(x_train,y_train,epochs=2,use_multiprocessing=True)

Epoch 1/2
126/126 [==============================] - 217s 2s/step - loss: 1.5086 - accuracy: 0.4442
Epoch 2/2
126/126 [==============================] - 219s 2s/step - loss: 0.9615 - accuracy: 0.6770


In [16]:
# saving model
save_path = input("Enter path(save model): ")
model.save(save_path)




Enter path(save model): new_transfer_learning
INFO:tensorflow:Assets written to: new_transfer_learning\assets


In [17]:
from keras.models import load_model
model_2 = load_model("new_transfer_learning/")

In [21]:
path = input("Enter folder path: ")
x_final,Y, Y_label,count = image_processing(path)


x_train,x_test,y_train,y_test = train_test_split(x_final,Y,random_state=0)

Enter folder path: ./simpsons/transfer_learning
comic_book_guy
disco_stu
edna_krabappel
fat_tony
gil
groundskeeper_willie
homer_simpson


In [ ]:
model_2.layers

In [22]:
for i in range(len(model_2.layers)-1):
    model_2.layers[i].trainable = False

In [23]:
new = model_2.layers[len(model_2.layers)-2].output
new = layers.Dense(32)(new)
new = layers.Dense(count)(new)
new_model = tf.keras.Model(inputs = model_2.input,outputs = new)

In [24]:
new_model.compile(optimizer='adam',
             loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
        )

In [26]:
hist_2 = new_model.fit(x_train,y_train,epochs=2,validation_split=0.2,use_multiprocessing=True)

Epoch 1/2
63/63 [==============================] - 28s 449ms/step - loss: 1.1411 - accuracy: 0.6262 - val_loss: 0.8638 - val_accuracy: 0.7083
Epoch 2/2
63/63 [==============================] - 32s 507ms/step - loss: 0.9047 - accuracy: 0.6819 - val_loss: 0.8263 - val_accuracy: 0.7123
